# Autocomplétion et génération de phrases avec des modèles de langue n-grammes

In [ ]:
import math
import random
import numpy as np
import pandas as pd
import nltk
import re
nltk.download('punkt')
nltk.data.path.append('.')

<a name='1'></a>
## 1.  Chargement et prétraitement des données (12 points)

<a name='1.1'></a>
### 1.1 Chargement des données (1 point)

Les données que vous allez utiliser dans ce travail sont contenues dans le fichier [trump.txt](./trump.txt)

Lisez le contenu de ce fichier et stockez-le dans une variable data.

Affichez ensuite les 300 premiers caractères



In [2]:
with open("trump.txt", "r", encoding='utf-8') as f:
    data = f.read()

display(data[0:300])

'Thank you very much.\nWe had an amazing convention.\nThat was one of the best.\nI think it was one of the best ever.\nIn terms -- in terms of enthusiasm, in terms of I think what it represents, getting our word out.\nIvanka was incredible last night.\nShe did an incredible job.\nAnd so many of the speakers'

<a name='1.2'></a>
### 1.2  Segmentation (2 points)

Pré-traitez les données en suivant les étapes suivantes:

1. Enlever les majuscules.
2. Remplacer les "\n" par des espaces
3. Séparer les données en phrases en utilisant les délimiteurs suivants `.`, `?` et `!` comme séparateur.
4. Enlever les signes de ponctuation (Attention de garder les espaces).
5. Enlever les phrases vides.
6. Segmenter les phrases avec la fonction nltk.word_tokenize()

Utilisez ensuite votre fonction pour prétraiter le jeu de données.

In [3]:

def preprocess(data):
    
    data = data.lower()
    
    sentences = re.split(r'\.|\?|\!', data.replace('\n', ' '))

    sentences = [re.sub('[^A-Za-z0-9 ]+', '', s).strip() for s in sentences]

    sentences = [s for s in sentences if len(s) > 0]

    sentences = [nltk.word_tokenize(sentence) for sentence in sentences] 

    return sentences

In [4]:
data = preprocess(data)

In [5]:
# test 
x = "Cats are independent.\nDogs are faithful."
preprocess(x)

[['cats', 'are', 'independent'], ['dogs', 'are', 'faithful']]

##### Sortie attendue

```CPP
[['cats', 'are', 'independent'], ['dogs', 'are', 'faithful']]
```

<a name='1.3'></a>
###  1.3 Création d'ensembles d'entraînement et de test (1 point)

Échantilloner de manière **aléatoire** 80% des données pour l'ensemble d'entrainement. Garder 20% pour l'ensemble de test. Utilisez la fonction train_test_split de sklearn. Stocker les résultats dans des variables.

In [7]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

### 1.4 Construction du vocabulaire (4 points)

Comme dans le TP1, construisez un vocabulaire à partir des données d'entraînement. Vous pouvez reprendre votre code du TP1.

Complétez la fonction **build_voc** qui retourne une liste de jetons qui sont présents au moins n fois (threshold passé en paramètre) dans la liste d'exemples (également passée en paramètre). Vous pouvez utiliser la classe collections.Counter.

Ensuite, appelez cette fonction pour construire votre vocabulaire à partir de l'ensemble d'entraînement **en utilisant threshold=2**. Imprimez la taille du vocabulaire.


In [8]:
from collections import Counter 

def build_voc(documents, threshold):
    vocabulary = Counter([word for sentence in documents for word in sentence ])
    return set([word for word, n_occurence in vocabulary.items() if n_occurence >= threshold])

voc = build_voc(train_data, 2)

print(len(voc))

5597


<a name='1.4'></a>
### 1.5 Mots hors vocabulaire (4 points)

Si votre modèle réalise de l'autocomplétion, mais qu'il rencontre un mot qu'il n'a jamais vu lors de l'entraînement, le modèle ne pourra donc pas prédire le mot suivant car il n'y a pas d'occurrence pour le mot actuel.

Ces mots sont appelés les mots hors vocabulaire (Out of Vocabulary) <b>OOV</b>.
Le pourcentage de mots inconnus dans l'ensemble de test est appelé le taux de mots <b> OOV </b>.

Pour gérer les mots inconnus lors de la prédiction, utilisez un jeton spécial 'unk' pour représenter tous les mots inconnus. Plus spécifiquement, la technique que vous utiliserez sera la suivante:

Complétez la fonction replace_oov qui convertit tous les mots qui ne font pas partie du vocabulaire en jeton '\<unk\>'.

Appelez ensuite votre fonction sur votre corpus d'entraînement et de test.



In [9]:
from collections import Counter

def replace_oov(tokenized_sentences, voc):
    
    new_data = [[word if word in voc else '<unk>' for word in sentence] for sentence in tokenized_sentences]
    
    return new_data

In [10]:
train_data = replace_oov(train_data, voc)
test_data = replace_oov(test_data, voc)

In [11]:
tokenized_sentences = [["cats", "sleep"], ["mice", "eat"], ["cats", "and", "mice"]]
vocabulary = build_voc([["cats", "sleep"], ["mice", "eat"], ["cats", "and", "mice"]], 2)
tmp_replaced_tokenized_sentences = replace_oov(tokenized_sentences, vocabulary)
print(f"Phrase initiale:")
print(tokenized_sentences)
print(f"Phrase segmentée avec'<unk>':")
print(tmp_replaced_tokenized_sentences)

Phrase initiale:
[['cats', 'sleep'], ['mice', 'eat'], ['cats', 'and', 'mice']]
Phrase segmentée avec'<unk>':
[['cats', '<unk>'], ['mice', '<unk>'], ['cats', '<unk>', 'mice']]


##### Sortie attendue
```CPP
Phrase initiale:
[['cats', 'sleep'], ['mice', 'eat'], ['cats', 'and', 'mice']]
Phrase segmentée avec '<unk>':
[['cats', '<unk>'], ['mice', '<unk>'], ['cats', '<unk>', 'mice']]
```

<a name='2'></a>
## 2. Modèles de langue n-gramme (18 points)



Dans cette section, vous développerez un modèle de langue n-grammes. Nous allons utiliser la formule: 

$$ \hat{P}(w_t | w_{t-1}\dots w_{t-n}) = \frac{C(w_{t-1}\dots w_{t-n}, w_t)}{C(w_{t-1}\dots w_{t-n})} \tag{2} $$

- La fonction $C(\cdots)$ représente le nombre d'occurrences de la séquence donnée.
- $\hat{P}$ signifie l'estimation de $P$.

Vous pouvez estimer cette probabilité en comptant les occurrences de ces séquences de mots dans les données d'entraînement.


<a name='2.1'></a>
### 2.1 Fréquence des n-grammes (3 points)



Vous allez commencer par mettre en œuvre une fonction qui calcule la fréquence des n-grammes pour un nombre arbitraire $n$.

Vous devez pré-traiter la phrase en ajoutant $n-1$ marqueurs de début de phrase "\<s\>" pour indiquer le commencement de la phrase.

- Par exemple, dans un modèle trigramme (N=3), une séquence avec deux jetons de début "\<s\>\<s\>" devrait prédire le premier mot d'une phrase. Ainsi, si la phrase est "J'aime la nourriture", modifiez-la pour devenir "\<s\>\<s\> J'aime la nourriture".
- Ajoutez aussi un jeton de fin "\<e\>" pour que le modèle puisse prédire quand terminer une phrase.
    

Dans cette implémentation, vous devez stocker les occurrences des n-grammes sous forme de dictionnaire.

- La clé de chaque paire clé-valeur dans le dictionnaire est un tuple de n mots (et non une liste).
- La valeur dans la paire clé-valeur est le nombre d'occurrences.

In [12]:

def count_n_grams(data, n, start_token='<s>', end_token = '<e>'):
    n_grams = {}

    for sentence in data:
        
        sentence = [start_token] * n + sentence + [end_token]

        for i in range((len(sentence) - (n-1) )): # complete this line

            n_gram = tuple(sentence[i:i+n])
            
            if n_gram in n_grams.keys():
            
                n_grams[n_gram] += 1
            else:
                n_grams[n_gram] = 1
    

    return n_grams

In [13]:
sentences = [['i', 'have', 'a', 'mouse'],
             ['this', 'mouse', 'likes', 'cats']]
print("Unigrammes:")
print(count_n_grams(sentences, 1))
print("Bigrammes:")
print(count_n_grams(sentences, 2))

Unigrammes:
{('<s>',): 2, ('i',): 1, ('have',): 1, ('a',): 1, ('mouse',): 2, ('<e>',): 2, ('this',): 1, ('likes',): 1, ('cats',): 1}
Bigrammes:
{('<s>', '<s>'): 2, ('<s>', 'i'): 1, ('i', 'have'): 1, ('have', 'a'): 1, ('a', 'mouse'): 1, ('mouse', '<e>'): 1, ('<s>', 'this'): 1, ('this', 'mouse'): 1, ('mouse', 'likes'): 1, ('likes', 'cats'): 1, ('cats', '<e>'): 1}


Sortie attendue:

```CPP
Unigrammes:
{('<s>',): 2, ('i',): 1, ('have',): 1, ('a',): 1, ('mouse',): 2, ('<e>',): 2, ('this',): 1, ('likes',): 1, ('cats',): 1}
Bigrammes:
{('<s>', '<s>'): 2, ('<s>', 'i'): 1, ('i', 'have'): 1, ('have', 'a'): 1, ('a', 'mouse'): 1, ('mouse', '<e>'): 1, ('<s>', 'this'): 1, ('this', 'mouse'): 1, ('mouse', 'likes'): 1, ('likes', 'cats'): 1, ('cats', '<e>'): 1}

```

<a name='2.2'></a>
### 2.2 Estimé du maximum de vraisemblance MLE (4 points)


Ensuite, estimez la probabilité d'un mot étant donnés les 'n' mots précédents avec les fréquences obtenues.

$$ \hat{P}(w_t | w_{t-1}\dots w_{t-n}) = \frac{C(w_{t-1}\dots w_{t-n}, w_t)}{C(w_{t-1}\dots w_{t-n})} \tag{2}$$ 


La fonction prend en entrée: 

- word : le mot dont on veut estimer la probabilité
- previous_n_gram : le n-gramme précédent, sous forme de tuple
- n_gram_counts: Un dictionnaire où la clé est le n-gramme et la valeur est la fréquence de ce n-gramme.
- n_plus1_gram_counts: Un autre dictionnaire, que vous utiliserez pour trouver la fréquence du n-gramme précédent plus le mot actuel.


In [14]:

def estimate_probability(word, previous_n_gram, n_gram_counts, n_plus1_gram_counts):
    
    n = len(list(n_gram_counts.keys())[0])
    
    previous_n_gram = tuple(previous_n_gram)
    
    previous_n_gram_count = n_gram_counts[previous_n_gram] if previous_n_gram in n_gram_counts  else 0

    n_plus1_gram = previous_n_gram + (word,)

    n_plus1_gram_count = n_plus1_gram_counts[n_plus1_gram] if n_plus1_gram in n_plus1_gram_counts  else 0
    
    probability = n_plus1_gram_count / previous_n_gram_count
    
    return probability

a) Quel est le problème de cette fonction? Quelle embûche pourrait-on rencontrer? Répondre avec un exemple.

> *Entrez votre réponse ici*

<a name='2.3'></a>
### 2.3  Lissage add-k (4 points)

Vous allez maintenant modifier votre fonction précédente en utilisant le lissage add-k.

$$ \hat{P}(w_t | w_{t-1}\dots w_{t-n}) = \frac{C(w_{t-1}\dots w_{t-n}, w_n) + k}{C(w_{t-1}\dots w_{t-n}) + k|V|} \tag{3} $$

Recodez la fonction au numéro 2.2 en ajoutant une constante de lissage $k$ and la taille du vocabulaire en paramètres supplémentaires. 

In [15]:

def estimate_probability_smoothing(word, previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=1.0):

    n = len(next(iter(n_gram_counts)))
    
    context = tuple(previous_n_gram)
    context_count = n_gram_counts[context] if context in n_gram_counts else 0
    denominator = context_count + k * vocabulary_size

    word_with_context = context + (word,)
    word_with_context_count = n_plus1_gram_counts[word_with_context] if word_with_context in n_plus1_gram_counts  else 0
    numerator = word_with_context_count + k
    
    probability = numerator / denominator
    
    return probability

In [16]:
# test
sentences = [['i', 'have', 'a', 'mouse'],
             ['this', 'mouse', 'likes', 'cats']]
unique_words = list(set(sentences[0] + sentences[1]))

bigram_counts = count_n_grams(sentences, 2)
trigram_counts = count_n_grams(sentences, 3)
tmp_prob = estimate_probability_smoothing("have", ['<s>', 'i'], bigram_counts, trigram_counts, len(unique_words), k=1)

print(f" La probabilité de 'have' étant donné le mot précédent 'i' est: {tmp_prob:.4f}")

 La probabilité de 'have' étant donné le mot précédent 'i' est: 0.2500


##### Sortie attendue

```CPP
 La probabilité de 'have' étant donné le mot précédent 'i' est: 0.2500
```

<a name='2.4'></a>
### 2.4 Calcul des probabilités des n-grammes

#### 2.4.1. Estimation des probabilités (4 points) 
Complétez la fonction estimate_probabilities qui calcule pour chaque mot du vocabulaire la probabilité d'être généré en utilisant la fonction avec lissage add-k. 

N'oubliez pas d'ajouter le jetons spécial "\<e\>" au vocabulaire

Cette fonction prends en entrée:
- previous_n_gram: le n-gramme précédent, sous forme de tuple
- n_gram_counts: Un dictionnaire où la clé est le n-gramme et la valeur est la fréquence de ce n-gramme.
- n_plus1_gram_counts: Un autre dictionnaire, que vous utiliserez pour trouver la fréquence du n-gramme précédent plus le mot actuel.
- vocabulary: l'ensemble du vocabulaire
- k: la constante de lissage

La fonction retourne un dictionnaire ayant pour clés tous les mots du vocabulaire ainsi que leur probabilité d'être générés

In [17]:
def estimate_probabilities(previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0):


    previous_n_gram = tuple(previous_n_gram)
    

    vocabulary = vocabulary + ["<e>"]
    
    probabilities = {}
    for word in vocabulary:
        probability = estimate_probability_smoothing(word, previous_n_gram, 
                                           n_gram_counts, n_plus1_gram_counts, 
                                           len(vocabulary), k=k)
        probabilities[word] = probability
    
    return probabilities

In [18]:
# test 
sentences = [['i', 'have', 'a', 'mouse'],
             ['this', 'mouse', 'likes', 'cats']]
unique_words = list(set(sentences[0] + sentences[1]))
unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)
estimate_probabilities(["a"], unigram_counts, bigram_counts, unique_words, k=1)

{'likes': 0.1111111111111111,
 'a': 0.1111111111111111,
 'cats': 0.1111111111111111,
 'i': 0.1111111111111111,
 'have': 0.1111111111111111,
 'this': 0.1111111111111111,
 'mouse': 0.2222222222222222,
 '<e>': 0.1111111111111111}

##### Sortie attendue

```CPP
{'likes': 0.1111111111111111,
 'have': 0.1111111111111111,
 'this': 0.1111111111111111,
 'i': 0.1111111111111111,
 'mouse': 0.2222222222222222,
 'a': 0.1111111111111111,
 'cats': 0.1111111111111111,
 '<e>': 0.1111111111111111}
```

#### 2.4.2. Probabilités étant donné un contexte (3 points)

Affichez maintenant les probabilités des tri-grammes étant donné le context "i will" en utilisant les données d'entraînement . N'affichez que les 10 mots les plus probables en ordre décroissant de probabilité.

In [42]:


bigram_counts_train = count_n_grams(train_data, 2)
trigram_counts_train = count_n_grams(train_data, 3)

probs_given_and = estimate_probabilities(['i', 'will'], bigram_counts_train, trigram_counts_train, list(voc), k=1)
display(sorted(probs_given_and.items(), key=lambda x: 1 - x[1])[0:10])

[('tell', 0.0070981916511745815),
 ('fix', 0.005239141456819334),
 ('fight', 0.005239141456819334),
 ('be', 0.0050701368936961295),
 ('never', 0.004225114078080108),
 ('say', 0.003718100388710495),
 ('not', 0.0025350684468480648),
 ('ask', 0.0023660638837248605),
 ('also', 0.0018590501943552475),
 ('work', 0.001521041068108839)]

<a name='4'></a>
## 3. Perplexité (15 points)

Dans cette section, vous allez générer le score de perplexité pour évaluer votre modèle sur l'ensemble de test.

Pour calculer le score de perplexité de l'ensemble de test sur un modèle n-gramme, utilisez :

$$PP(W) =\sqrt[N]{ \prod_{t=n}^{N} \frac{1}{P(w_t | w_{t-n} \cdots w_{t-1})} } \tag{4.1}$$

Plus les probabilités sont élevées, plus la perplexité sera basse. 

### 3.1. Calcul de la perplexité (4 points)
Complétez la fonction `calculate_perplexity`, qui pour une phrase donnée, nous donne le score de perplexité. Cette fonction prend en entrée:


- sentence: La phrase pour laquelle vous devez calculer la perplexité
- n_gram_counts: Un dictionnaire où la clé est le n-gramme et la valeur est la fréquence de ce n-gramme.
- n_plus1_gram_counts: Un autre dictionnaire, que vous utiliserez pour trouver la fréquence du n-gramme précédent plus le mot actuel.
- vocabulary_size: la taille du vocabulaire
- k: la constante de lissage

In [61]:

def calculate_perplexity(sentence, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=1.0):
   
    n = len(list(n_gram_counts.keys())[0]) 
    
    N = len(sentence) + 1
    
    sentence = ["<s>"] * n + sentence + ["<e>"]
    
    sentence = tuple(sentence)
    
    product_pi = 1.0
    
    for t in range(n, len(sentence)): 
        
        n_gram = sentence[t-n:t]
        
        word = sentence[t]    
        
        probability = estimate_probability_smoothing(word, n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=k)
        product_pi *= (1 / probability)**(1/N)
        
        
    
    
    return product_pi

In [62]:
# test 

sentences = [['i', 'have', 'a', 'mouse'],
             ['this', 'mouse', 'likes', 'cats']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)


perplexity = calculate_perplexity(sentences[0],
                                         unigram_counts, bigram_counts,
                                         len(unique_words), k=1.0)
print(f"Perplexité de la première phrase: {perplexity:.4f}")


Perplexité de la première phrase: 4.1930


### 3.2. Perplexité sur une phrase d'entraînement (4 points)
Calculez et affichez la perplexité des modèles bi-grammes, tri-grammes et quadri-grammes à l'aide de votre fonction `calculate_perplexity` définie plus haut sur la première phrase de votre corpus d'entraînement. Utilisez K=0.01 ici.

In [63]:

unigram_counts_train = count_n_grams(train_data, 1)
bigram_counts_train = count_n_grams(train_data, 2)
trigram_counts_train = count_n_grams(train_data, 3)
quadgram_counts_train = count_n_grams(train_data, 4)
#qintgram_counts_train = count_n_grams(train_data, 5)

perplexity_bigram = calculate_perplexity(train_data[0],
                                         unigram_counts_train, bigram_counts_train,
                                         len(voc), k=0.01)
print(f"Perplexité du modele 2-gramme : {perplexity_bigram:.4f}")

perplexity_trigram = calculate_perplexity(train_data[0],
                                         bigram_counts_train, trigram_counts_train,
                                         len(voc), k=0.01)
print(f"Perplexité du modele 3-gramme : {perplexity_trigram:.4f}")

perplexity_quadgram = calculate_perplexity(train_data[0],
                                         trigram_counts_train, quadgram_counts_train,
                                         len(voc), k=0.01)
print(f"Perplexité du modele 4-gramme : {perplexity_quadgram:.4f}")


Perplexité du modele 2-gramme : 71.9584
Perplexité du modele 3-gramme : 50.7269
Perplexité du modele 4-gramme : 50.6769


### 3.3. Perplexité du corpus de test

#### 3.3.1. Vous pouvez maintenant calculer et afficher la perplexité des modèles bi-grammes, tri-grammes et quadri-grammes sur votre corpus de test. K=1 ici. (4 points)

Pour calculer la perplexité d'un corpus de *m* phrases, il suffit de suivre la formule suivante : 

Soit *N* le nombre total de jetons dans le corpus de test C.

$$Perplexity(C) = \Big(\frac{1}{P(s_1, ..., s_m)}\Big)^{1/N}$$ 
$$P(s_1, ..., s_m) = \prod_{i=1}^{m} p(s_i)$$
$$p(s_i) = \prod_{t=1}^{n} \hat{P}(w_t | w_{t-1}\dots w_{t-n})$$

Puisqu'il s'agit d'un multiplication de probabilités (situées entre 0 et 1), le produit devient nul très rapidement. C'est pourquoi il est plus efficace d'effectuer une transformation vers un espace logarithmique pour transformer les multiplications en addition. Cela donne ainsi la formule suivante:

$$LogPerplexity(C) = 2^{-\frac{1}{N} \sum_{k=1}^{m} log_{2} \; p(s_k)}$$ 


In [24]:
def calculate_perplexity_corpus(corpus, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=1.0):
    
    n = len(list(n_gram_counts.keys())[0]) 
    
    sum_sentence = 0.0
    N = 0
    for sentence in corpus:
        sentence = ["<s>"] * (n) + sentence + ["<e>"]
        N += len(sentence) - n

        for t in range(n, len(sentence)): # complete this line
            
            n_gram = sentence[t-n:t]
            word = sentence[t]

            probability = estimate_probability_smoothing(word, n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=k)
            
            sum_sentence += math.log2(probability)
    
    perplexity = 2 ** ((-1 / N) * sum_sentence)
    
    return perplexity


In [25]:
n_gram_counts = {('<s>', 'quick'): 1, ('the', 'quick'): 1, ('quick', 'brown'): 1, ('brown', 'fox'): 1, ('jumps', 'over'): 1, ('over', 'the'): 1, ('the', 'lazy'): 1, ('lazy', 'dog'): 1, ('dog', '<e>'): 1}
n_plus1_gram_counts = { ('<s>', '<s>', 'the', ): 1, ('<s>', 'the', 'quick'): 1, ('the', 'quick', 'brown'): 1, ('quick', 'brown', 'fox'): 1, ('jumps', 'over', 'the'): 1, ('over', 'the', 'lazy'): 1, ('the', 'lazy', 'dog'): 1, ('lazy', 'dog', '<e>'): 1}

train_corpus = [["the", "quick", "brown", "fox"], ["jumps", "over", "the", "lazy", "dog"]]
n_gram_counts = {('<s>', '<s>'): 2, ('<s>', 'the'): 1, ('<s>', 'jumps'): 1, ('the', 'quick'): 1, ('quick', 'brown'): 1, ('brown', 'fox'): 1, ('fox', '<e>'): 1, ('jumps', 'over'): 1, ('over', 'the'): 1, ('the', 'lazy'): 1, ('lazy', 'dog'): 1, ('dog', '<e>'): 1}
n_plus1_gram_counts = {('<s>', '<s>', '<s>', ): 2, ('<s>', '<s>', 'the', ): 1, ('<s>', 'the', 'quick'): 1,  ('<s>', '<s>', 'jumps', ): 1, ('<s>', 'jumps', 'over'): 1, ('the', 'quick', 'brown'): 1, ('quick', 'brown', 'fox'): 1, ('brown', 'fox', '<e>'): 1, ('jumps', 'over', 'the'): 1, ('over', 'the', 'lazy'): 1, ('the', 'lazy', 'dog'): 1, ('lazy', 'dog', '<e>'): 1}

vocabulary = ["the", "quick", "brown", "fox", "jumps", "over", "lazy", "dog", "<e>"]

N = 4
V = len(vocabulary)

test_corpus = [["the", "fox"], ["jumps"]]

# pour k=1
p_the = math.log2((1. + 1.)/(2. + 1.* V))   # prob. pour "the" avec ('<s>', '<s>')
p_fox = math.log2((0. + 1.)/(1. + 1.* V))   # prob. pour "fox" avec ('<s>', 'the')
p_e = math.log2((0. + 1.)/(0. + 1.* V))     # prob. pour "<e>" avec ('the', 'fox')

p_jumps = math.log2((1. + 1.)/(2. + 1.* V)) # prob. pour "jumps" avec ('<s>', '<s>')
p_e2 = math.log2((0. + 1.)/(1. + 1.* V))    # prob. pour "<e>" avec ('<s>', 'jumps')



expected_perplexity = p_the + p_fox + p_e + p_jumps + p_e2

expected_perplexity =  2 ** ((-1 / N) * expected_perplexity)


# Call the calculate_perplexity_corpus function
calculated_perplexity = calculate_perplexity_corpus(test_corpus, n_gram_counts, n_plus1_gram_counts, len(vocabulary), k=1.0)

print("Perplexité du corpus de test: ", calculated_perplexity)

Perplexité du corpus de test:  7.708920690856638


#### Sortie attendue

    Perplexité du corpus de test:  7.708920690856638

In [65]:


max_grams = 4
grams = [count_n_grams(train_data, i) for i in range(1, max_grams+1)]

for i in range(len(grams) - 1):

    n_gram = grams[i]
    n_plus1_gram = grams[i+1]
    perplexity = calculate_perplexity_corpus(test_data, n_gram, n_plus1_gram, len(voc), k=0.01)
    
    print(f'Perplexité du modèle {i + 2}-gramme : {perplexity:.4f}')


Perplexité du modèle 2-gramme : 98.9474
Perplexité du modèle 3-gramme : 208.9466
Perplexité du modèle 4-gramme : 489.4859


#### 3.3.2. Les perplexités attendues peuvent sembler contre-intuitives.  Comparez-les aux perplexités obtenues sur l'ensemble d'entrainement pour les mêmes modèles. Comment expliquez-vous ces résultats et quelle est votre conclusion ?  (3 points)

> *Entrez votre réponse ici*

<a name='5'></a>
## 4. Construction d'un modèle d'autocomplétion (15 points)

Dans cette dernière partie, vous allez utiliser les modèles n-grammes construits aux numéros précédents afin de faire un modèle d'autocomplétion.

<a name='5.1'></a>
### 4.1 Suggestion d'un mot à partir d'un préfixe (5 points)


La première étape sera de construire une fonction qui suggère un mot à partir des premiers caractères entrés par un utilisateur, considérant un seul type de n-gramme.  

Complétez la fonction `suggest_word` qui calcule les probabilités pour tous les mots suivants possibles et suggère le mot le plus probable. Comme contrainte supplémentaire, le mot suggéré doit commencer avec le préfixe passé en paramètre. Utilisez vos fonctions provenant du numéro 2. (Modèle n-gramme de mots) pour faire vos prédictions.

Cette fonction prends en paramètre:
- previous_n_gram: le n-gramme précédent, sous forme de tuple
- n_gram_counts: Un dictionnaire où la clé est le n-gramme et la valeur est la fréquence de ce n-gramme.
- n_plus1_gram_counts: Un autre dictionnaire, que vous utiliserez pour trouver la fréquence du n-gramme précédent plus le mot actuel.
- vocabulary_size: la taille du vocabulaire
- k: la constante de lissage
- prefixe: Le début du mot que l'on veut prédire

Elle retourne le mot le plus probable avec la probabilité associée

In [27]:
import numpy as np
import time
from random import choices

def suggest_word(previous_tokens, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0, prefixe=""):
    
    n = len(list(n_gram_counts.keys())[0]) 

    previous_n_gram = previous_tokens[-n:]
    
    probabilities = estimate_probabilities(previous_n_gram,
                                           n_gram_counts, n_plus1_gram_counts,
                                           vocabulary, k=k)
    word_probs = [(word,prob) for word, prob in probabilities.items() if word.startswith(prefixe)]
    
    if len(word_probs) == 0:
        return prefixe, 1
    
    words, probs = zip(*word_probs)
    
    best_word_index = np.argmax(probs)
    
    return words[best_word_index], probs[best_word_index]

In [28]:
# test
sentences = [['i', 'have', 'a', 'mouse'],
             ['this', 'mouse', 'likes', 'cats']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)

previous_tokens = ["i", "have"]
tmp_suggest1 = suggest_word(previous_tokens, unigram_counts, bigram_counts, unique_words, k=1.0)
print(f" avec les mots précédents 'i have',\n\t le mot suggéré est `{tmp_suggest1[0]}` avec la probabilité {tmp_suggest1[1]:.4f}")

print()


tmp_starts_with = 'm'
tmp_suggest2 = suggest_word(previous_tokens, unigram_counts, bigram_counts, unique_words, k=1.0, prefixe=tmp_starts_with)
print(f"avec les mots précédents 'i have', et une suggestion qui commence par `{tmp_starts_with}`\n\t le mot suggéré est : `{tmp_suggest2[0]}` avec une probabilité de {tmp_suggest2[1]:.4f}")

 avec les mots précédents 'i have',
	 le mot suggéré est `a` avec la probabilité 0.2222

avec les mots précédents 'i have', et une suggestion qui commence par `m`
	 le mot suggéré est : `mouse` avec une probabilité de 0.1111


### Sortie attendue

```CPP
avec les mots précédents 'i have',
	 le mot suggéré est `a` avec la probabilité 0.2222

avec les mots précédents 'i have', et une suggestion qui commence par `m`
	 le mot suggéré est : `mouse` avec une probabilité de 0.1111

```

<a name='5.2'></a>
### 4.2 Suggestions multiples (5 points)

Afin de suggérer plusieurs mots à l'utilisateur, une stratégie que l'on peut utiliser est de retourner un ensemble de mots suggérés par plusieurs types de modèles n-grammes.

En utilisant la fonction `suggest_word` du numéro précédent, complétez la fonction `get_suggestions` qui retourne les suggestions des modèles n-grammes passés en paramètre. Vous devrez aussi enlever les doublons dans les suggestions s'il y en a, et ordonner la liste des suggestions en commençant par le mot ayant la probabilité la plus élevée.

La fonction get_suggestions prends en paramètres:
- previous_n_gram: le n-gramme précédent, sous forme de tuple
- n_gram_counts_list: une liste de n-grammes dans l'ordre suivant [unigrammes, bigrammes, trigrammes, quadrigrammes, ...]
- vocabulary_size: la taille du vocabulaire
- k: la constante de lissage (entre 0 et 1)
- prefixe: Le début du mot que l'on veut prédire, "" si au aucun préfixe

In [29]:
def get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0, prefixe=""):
    model_counts = len(n_gram_counts_list)
    suggestions = []
    for i in range(model_counts-1):
        n_gram_counts = n_gram_counts_list[i]
        n_plus1_gram_counts = n_gram_counts_list[i+1]

        # Making sure the output is valid given the n-gram
        if len(previous_tokens) - 2 >= i:
            # If input is longer than size of n-gram, reduce it
            formatted_previous = previous_tokens[-(max(0, i+1)):]
        else:
            # If input is shorter, add <s> token
            formatted_previous = ['<s>'] * max(0, i - len(previous_tokens) + 1) + previous_tokens
        
        suggestion = suggest_word(formatted_previous, n_gram_counts,
                                    n_plus1_gram_counts, vocabulary,
                                    k=k, prefixe=prefixe)
        suggestions.append(suggestion)
    
    suggestions = list(map(lambda x: x[0] , sorted(suggestions, key=lambda x: -x[1])))
    return list(set(suggestions))

In [30]:
# test 
sentences = [['i', 'have', 'a', 'mouse'],
             ['this', 'mouse', 'likes', 'cats']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)
trigram_counts = count_n_grams(sentences, 3)
quadgram_counts = count_n_grams(sentences, 4)
qintgram_counts = count_n_grams(sentences, 5)

n_gram_counts_list = [unigram_counts, bigram_counts, trigram_counts, quadgram_counts, qintgram_counts]
previous_tokens = ["i", "have"]
tmp_suggest3 = get_suggestions(previous_tokens, n_gram_counts_list, unique_words, k=1.0)

print(f"Etant donné les mots i have, je suggère :")
display(tmp_suggest3)

Etant donné les mots i have, je suggère :


['a']

##### Sortie attendue

```CPP 
Etant donné les mots i have, je suggère :
['a']
```

<a name='5.3'></a>
### 4.3 Autocomplétion (5 points)

Il est maintenant temps de combiner vos fonctions afin de créer le modèle d'autocomplétion. En utilisant le jeu de données d'entraînement, calculez la fréquence des n-grammes allant de 1 à 5 et utilisez la fonction *get_suggestions* afin de suggérer des mots. Vous devrez être en mesure de toujours suggérer des mots à partir du dernier mot entré par l'utilisateur.

Complétez la fonction *update_suggestions*:
- la variable texte_actuel contient tout le texte entré par l'utilisateur
- la variable top_suggestions contient les suggestions qui seront proposées

Vous devrez changer le contenu de la variable top_suggestions pour qu'elle contienne les suggestions des n-grammes.

In [31]:
unigram_counts = count_n_grams(train_data, 1)
bigram_counts = count_n_grams(train_data, 2)
trigram_counts = count_n_grams(train_data, 3)
quadgram_counts = count_n_grams(train_data, 4)
qintgram_counts = count_n_grams(train_data, 5)

n_gram_counts_list_trained = [unigram_counts, bigram_counts, trigram_counts, quadgram_counts, qintgram_counts]

In [32]:
import ipywidgets as widgets
from IPython.display import display


text_input = widgets.Text(placeholder="Entrez votre text ici...")

suggestions_label = widgets.Label(value="Suggestions: ")

def update_suggestions(change):
    texte_actuel = change["new"].lower()
    
    prefixe = texte_actuel.split(" ")[-1]
    previous_tokens = texte_actuel.split(" ")[:-1]

    top_suggestions = get_suggestions(previous_tokens, n_gram_counts_list_trained, list(voc), k=0.01, prefixe=prefixe)
    suggestions_label.value = "Suggestions: " + ", ".join(top_suggestions)

text_input.observe(update_suggestions, names="value")

display(text_input)
display(suggestions_label)

Text(value='', placeholder='Entrez votre text ici...')

Label(value='Suggestions: ')

<a name='6'></a>
## 5. Modèle de génération de phrases (30 points)

Il est aussi possible de construire un modèle de génération de phrases avec les modèles n-grammes. 


#### Dans la cadre d'un modèle de génération de phrases, indiquez pourquoi la stratégie de suggestion des mots en 5.1 ne peut pas fonctionner ? (3 points)

>La fonction *suggest_word*, retourne le mot ayant la plus grande probabilité d'être généré. nous allons toujours nous retrouver avec la même phrase générée.

<a name='6.1'></a>

### 5.1 Génération stochastique de mots (5 points)

Recodez la fonction suggest_word afin d'utiliser une suggestion stochastique. Autrement dit, au lieu de retourner le mot le plus probable, vous devrez générez le mot suivant selon sa probabilité.

Par exemple si le mot 'like' a la probabilité 0.25 d'être généré, alors il sera retourné 25% du temps.

```

In [33]:
import numpy as np
import time
from random import choices

def suggest_word_with_probs(previous_tokens, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0):
    
    n = len(list(n_gram_counts.keys())[0]) 

    previous_n_gram = previous_tokens[-n:]
    
    probabilities = estimate_probabilities(previous_n_gram,
                                           n_gram_counts, n_plus1_gram_counts,
                                           vocabulary, k=k)
    word_probs = [(word,prob) for word, prob in probabilities.items()]
    
    
    words, probs = zip(*word_probs)
    
    choice = choices(word_probs, probs)[0]

    return choice[0], choice[1] 


<a name='6.2'></a>
### 5.2 Générations de phrases (10 points)

#### 5.2.1. Génération stochastique (4 points)
Complétez maintenant la fonction `generate_sentence` qui génère une phrase longue de n_words en appelant votre nouvelle fonction `suggest_words_with_probs`. La génération doit s'arrêter si le modèle génère un jeton de fin de phrase.

Il ne faut pas oublier d'initialiser les phrases à générer avec le bon nombre de jetons de début de phrase (`<s>`). Par exemple, s'il s'agit d'un modèle bigramme, il faudra initialiser la phrase à [`<s>`]. S'il s'agit d'un modèle trigramme, il faudra initialiser la phrase à [`<s>`, `<s>`]. Vous pouvez trouver la taille du contexte à l'aide de l'expression suivante `len(next(iter(n_gram_counts)))`.

Ensuite, il faudra passer à la fonction `suggest_word` les `n` derniers mots générés où `n` correspond à la taille du contexte.
Finalement, il faudra arrêter la génération si le jeton généré est le jeton de fin (`<e>`)

In [34]:
def generate_sentence(n_words, n_gram_counts, n_plus1_gram_counts, vocabulary, k=0.0001):
    context_length = len(next(iter(n_gram_counts)))
    sentence = ['<s>'] * (context_length)
    context = sentence
    for i in range(n_words):

        suggestion = suggest_word_with_probs(context, n_gram_counts,
                                n_plus1_gram_counts, vocabulary,
                                k=k)
        sentence.append(suggestion[0])
        context = sentence[-context_length:]
        if sentence[-1] == "<e>":
            break

    return " ".join(list(sentence[context_length:-1]))


#### 5.2.2. Test sur des n-grammes (2 points)
Testez ensuite votre fonction avec des trigrammes et des 5-grammes.

In [35]:
print('5-grammes : \n\n')
for i in range(10):
    print(generate_sentence(15, quadgram_counts, qintgram_counts, list(voc), k=1))

5-grammes : 


youre raped theory involves point really behalf unfairness radical recommendations barely lose worked removal
its jackport lingering powered eliminate gun okeechobee ground ends designed super pledges better clintons
amount work kids hose sorry decisions collusion screening explanation brian yearning encouraged before credence
what by himself badlydepleted demonstrators outsider parttime priorities divider wont vile selling somali guard
you expect assimilation bond keeps dads award tougher status desire dallas singapore ballroom minds
reptile breakfast billions personal neighbors deliver closed don finisher halt products strongly spin admirals
you walk lone sitting farms taxation beneficiary minnesota interested unborn costing prepping class reaching
dont 77000 income amounts fierce overdue uphold graham restore eternal bleeding unlock nascar sicko
gon lie 150 consider los ally warm quoted elect multiple pilot entry fest welders
sadly hey thered mvp gates paul perfectly 

In [36]:
print('3-grammes : \n\n')

for i in range(10):
    print(generate_sentence(10, bigram_counts, trigram_counts, list(voc)))

3-grammes : 


are everyday tour general withdraw served employers burglary walls
its a terrible terrible breeding ground
all my life
right right
its amazing what percentage of very very well and
get out theyre getting massive amounts of money to
i am with you i ran against him
the middle east
because they know it yet but theyre going to
shed taxes whenever beyond good sleep brave capability came


#### 5.2.3. Avec k=1.0, que se passe-t-il avec les phrases générées et quelle en est la raison principale ? Que pouvez-vous faire pour améliorer la situation? (2 points)

> Les phrases ne sont pas vraiment cohérentes avec k=1.0 car on a trop transferré de masse de probabilité des événements fréquents vers les événements rares. 
On peut adopter un lissage moins brutal et descendre la valeur de k.

#### 5.2.4.  Quels sont les problèmes si la constante k a une valeur trop petite, voir 0?  (2 points)

> Avec k=0, on revient à un calcul de l'estimé de maximum de vraisemblance, ce qui entraine des possibilités de probabilités à 0 si le corpus de test contient des n-grammes inconnus et une incapacité donc à calculer la perplexité du modèle n-gramme. 

<a name='5.3'></a>
### 5.3. Amélioration de la génération stochastique de mots (12 points)

#### 5.3.1. Amélioration stochastique 

Comme vous avez pu l'observer, la génération stochastique, bien qu'elle soit efficace pour générer des phrases différentes, a tendance à ne pas générer des phrases toujours cohérentes. Proposez une amélioration de la méthode `suggest_word` que vous implémenterez dans la méthode `suggest_word_new` permettant de générer des phrases plus cohérentes. 

##### Décrivez votre méthode dans la cellule suivante (3 points)

> Il existe plusieurs implémentations possibles. Par exemple, ils peuvent utiliser les 10 mots les plus probables. Sinon, ils peuvent faire du backoff lorsque les probabilités sont nulles.

##### Implémentez la méthode proposée (5 points)

In [37]:
import numpy as np
import time
from random import choices

def suggest_word_new(previous_tokens, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0):

    n = len(list(n_gram_counts.keys())[0])

    previous_n_gram = previous_tokens[-n:]
    
    probabilities = estimate_probabilities(previous_n_gram,
                                           n_gram_counts, n_plus1_gram_counts,
                                           vocabulary, k=k)
    word_probs = [(word,prob) for word, prob in probabilities.items()]
    words, probs = zip(*word_probs)

    best_choices = np.argsort(probs)[-10:]

    best_words = []
    best_probs = []
    for i in best_choices:
        best_words.append(words[i])
        best_probs.append(probs[i])

    choice = choices(best_words, best_probs)

    return choice[0]

#### 5.3.2. Génération améliorée (2 points)
Recodez maintenant la fonction `generate_sentence_new` pour appeler votre nouvelle méthode `suggest_word_new`. 

In [38]:
def generate_sentence_new(n_words, n_gram_counts, n_plus1_gram_counts, vocabulary, k=0.001):
    context_length = len(next(iter(n_gram_counts)))
    sentence = ['<s>'] * (context_length)
    context = sentence
    for i in range(n_words):

        suggestion = suggest_word_new(context, n_gram_counts,
                                n_plus1_gram_counts, vocabulary,
                                k=k)
        sentence.append(suggestion)
        context = sentence[-context_length:]
        if sentence[-1] == "<e>":
            break

    return " ".join(list(sentence[context_length:-1]))
    

#### 5.3.3. Test sur des n-grammes (2 points)
Testez ensuite votre fonction avec des 3-grammes et des 5-grammes et validez que les phrases sont plus cohérentes.

In [39]:
print('5-grammes : \n\n')
for i in range(10):
    print(generate_sentence_new(15, quadgram_counts, qintgram_counts, list(voc), k=0.001))

5-grammes : 


its just as good but you need good strong hands
and i have to say the republican national convention was such a tremendous success
and its getting bigger very fast
its time we stop
were going have the wall
and i have a very special message for you tonight
but you could have picked a liberal person and we understand it we could
and i have to do with it
and i said to her last night i said to her yesterday i said
and i have to tell you


In [40]:
print('3-grammes : \n\n')

for i in range(10):
    print(generate_sentence_new(15, bigram_counts_train, trigram_counts_train, list(voc), k=0.001))

3-grammes : 


you dont know if hes here
so we got to get in
but they say well ok maybe because i know so well know
and i will fight harder for you than anyone theyve ever done it i
the national model of providing school choice
and i think the biggest crowds in the back of her workrelated emails
and by the way were going to be a very good job because our
but we have to do is were going to have a government of by
were going to do it
the other night right near fort bragg
